In [1]:
!pip install pyts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 9.6 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyts.image import GramianAngularField
from sklearn.preprocessing import StandardScaler
np.random.seed(46)
path = '/content/final_df.csv'

In [3]:
df = pd.read_csv(path)
protocol_states = ['Base', 'TSST']
df = df[df['labels'].isin(protocol_states)]
mapping_dict = {'Base': 0,
                'TSST': 1}
df['labels'] = df['labels'].map(mapping_dict)
ID = df['ID'].reset_index(drop=True) #store id column for later usage
df.drop(['Time'], inplace=True, axis = 1)
df.head()

,ID,hr,temp,labels
420,0,75.563333,35.447233,0
421,0,75.570000,35.549821,0
422,0,75.573333,35.449041,0
423,0,75.593333,35.543187,0
424,0,75.643333,35.443003,0


In [4]:
scaler = StandardScaler()
columns_to_scale = ['hr', 'temp']
scaled_df = df.copy(deep=True)
scaled_df[['hr', 'temp']] = scaler.fit_transform(scaled_df[['hr', 'temp']])
scaled_df.head()

,ID,hr,temp,labels
420,0,-0.084655,1.580124,0
421,0,-0.084002,1.639081,0
422,0,-0.083675,1.581163,0
423,0,-0.081715,1.635268,0
424,0,-0.076815,1.577693,0


In [5]:
output_folder = '/content/gaf_imgs'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def apply_gaf_transformation(data_array, filename):
    transformer = GramianAngularField(image_size=250, sample_range=(-1, 1))
    img = transformer.fit_transform(data_array)

    black = np.zeros((1, 250, 250), dtype=np.float32)
    gaf = np.concatenate((img, black), axis=0)

    scaled = (gaf - gaf.min()) / (gaf.max() - gaf.min())
    scaled = np.transpose(scaled, (1, 2, 0))

    plt.imsave(filename, scaled)

In [6]:
for i in scaled_df['ID'].unique():
    for label in [0, 1]:
        df = scaled_df[(scaled_df['ID'] == i) & (scaled_df['labels'] == label)]
        array = df[['hr', 'temp']].values
        array = np.transpose(array)

        label_folder = os.path.join(output_folder, f'label_{label}')
        if not os.path.exists(label_folder):
            os.makedirs(label_folder)

        filename = os.path.join(label_folder, f'img_{i}.png')

        apply_gaf_transformation(array, filename)

In [7]:
import shutil
folder_to_download = "/content/gaf_imgs"
shutil.make_archive("/content/gaf_imgs", 'zip', folder_to_download)

'/content/gaf_imgs.zip'

In [8]:
from google.colab import files
files.download('/content/gaf_imgs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>